In [2]:
import   pickle
import collections
import os
import numpy as np
import random 

In [3]:
infile = open('cv_split_5_fold_seizure_wise_v1.4.0.pkl','rb') #use the data prepared for cross validation
new_dict = pickle.load(infile)
infile.close()

seed = 0
random.seed(seed) # Reset the seed so all runs are the same.

source  = 'processed_2/v1.4.0/fft/fft_seizures_wl1_ws_0.25_sf_250_fft_min_1_fft_max_24/' #load the data from this folder


train_all = []    #read all pkl files from the folder
for i in range(5):
    train_all.append(new_dict[str(i+1)]['train']) 

train = list(set([item for sublist in train_all for item in sublist]))
print(len(train))

2009


In [4]:
#check if the training and validation sets can cover the whole dataset
val_all = []
for i in range(5):
    val_all.append(new_dict[str(i+1)]['val']) 

val = list(set([item for sublist in val_all for item in sublist]))
print(len(val))

2009


In [5]:
#check if training and validation sets are the same
train.sort() == val.sort()

True

In [6]:
#Calculate the samples in each category
table = {'ABSZ':0,'CPSZ':0, 'FNSZ':0, 'GNSZ':0, 'SPSZ':0, 'TCSZ':0, 'TNSZ':0}
for item in train:
    table[item[:-4][-4:]] += 1

In [7]:
table

{'ABSZ': 99,
 'CPSZ': 342,
 'FNSZ': 992,
 'GNSZ': 415,
 'SPSZ': 44,
 'TCSZ': 50,
 'TNSZ': 67}

In [8]:
#put the pkl files into the table below
table_samples = {'ABSZ':[],'CPSZ':[], 'FNSZ':[], 'GNSZ':[], 'SPSZ':[], 'TCSZ':[], 'TNSZ':[]}
for item in val:
    table_samples[item[:-4][-4:]].append(item)
len(table_samples['TNSZ'])

67

In [9]:
a, b = ['1', '2', '3'], ['1', '2']
set(a) - set(b)

{'3'}

In [10]:
#Use 20% samples in each category and combine them as the test set.
test_samples = []

for seizure_type in table_samples:
    all_sample = table_samples[seizure_type]
    test_sample = random.sample(all_sample, int(0.2*len(all_sample)))
    test_samples.append(test_sample)
    
test_all = [item for sublist in test_samples for item in sublist] 
print(len(test_all))
test_all

399


['szr_1962_pid_00008608_type_ABSZ.pkl',
 'szr_2010_pid_00002657_type_ABSZ.pkl',
 'szr_1966_pid_00001113_type_ABSZ.pkl',
 'szr_1918_pid_00000675_type_ABSZ.pkl',
 'szr_1946_pid_00001984_type_ABSZ.pkl',
 'szr_1978_pid_00001413_type_ABSZ.pkl',
 'szr_1975_pid_00001113_type_ABSZ.pkl',
 'szr_1964_pid_00001113_type_ABSZ.pkl',
 'szr_1951_pid_00003306_type_ABSZ.pkl',
 'szr_1974_pid_00001113_type_ABSZ.pkl',
 'szr_1958_pid_00003635_type_ABSZ.pkl',
 'szr_1987_pid_00001413_type_ABSZ.pkl',
 'szr_1940_pid_00001984_type_ABSZ.pkl',
 'szr_1977_pid_00001113_type_ABSZ.pkl',
 'szr_1930_pid_00000675_type_ABSZ.pkl',
 'szr_1949_pid_00003281_type_ABSZ.pkl',
 'szr_2009_pid_00002657_type_ABSZ.pkl',
 'szr_1925_pid_00000675_type_ABSZ.pkl',
 'szr_1992_pid_00002448_type_ABSZ.pkl',
 'szr_1170_pid_00006904_type_CPSZ.pkl',
 'szr_1314_pid_00005633_type_CPSZ.pkl',
 'szr_1350_pid_00006514_type_CPSZ.pkl',
 'szr_1117_pid_00002297_type_CPSZ.pkl',
 'szr_1200_pid_00004892_type_CPSZ.pkl',
 'szr_1092_pid_00008544_type_CPSZ.pkl',


In [11]:
table = {'ABSZ':0,'CPSZ':0, 'FNSZ':0, 'GNSZ':0, 'SPSZ':0, 'TCSZ':0, 'TNSZ':0}
for item in test_all:
    table[item[:-4][-4:]] += 1
    
table

{'ABSZ': 19,
 'CPSZ': 68,
 'FNSZ': 198,
 'GNSZ': 83,
 'SPSZ': 8,
 'TCSZ': 10,
 'TNSZ': 13}

In [40]:
for i in range(5):
    train = new_dict[str(i+1)]['train']   #read the train data for the 5-fold cross validation
    train = list(set(train) - set(test_all)) #

    
    val =  new_dict[str(i+1)]['val']      #read the validation data for the 5-fold cross validation
    val = list(set(val) - set(test_all))
    print(len(train), len(val), len(test_all))
    
    train_data, val_data = [],[]   #these two variables are used to store EEG signals from the pkl files
    train_label_before,train_label_after,val_label_before,val_label_after = [],[],[],[] #label_before variables are original labels in the pkl files,in label_after, the first axis is expended according to the first axis of train_data 
    train_id_before,train_id_after,val_id_before,val_id_after = [],[],[],[] #id_before variables are original patient ids,in id_after files, the first axis is expended according to the first axis of train_data 
    samples_train, samples_val = 0,0
    filename_train, filename_val = [], []
    
    for j in range(len(train)): #read train data, train label and patient id from the pkl files.
        filename_train.append(train[j])
        path_train = source + train[j]  #location of the pkl files
        sz = pickle.load(open(path_train, 'rb'))  #load the file
        samples_train  +=  np.shape(sz.data)[0] #the number of EEG clips in each pkl files
        #train_id_all.append(sz.patient_id)
        train_id_before.append(sz.patient_id)   #original patient id
        train_id_after.append([sz.patient_id]*np.shape(sz.data)[0])   #expended patient id according to the number of EEG clips in each pkl file
        train_label_before.append(sz.seizure_type)   #original label
        train_label_after.append([sz.seizure_type]*np.shape(sz.data)[0]) #expended training lables according to the number of EEG clips in each pkl file
        train_data.append(sz.data)
        
    for j in range(len(val)):  #read validation data, validation label and patient id from the pkl files.
        filename_val.append(val[j])
        path_val = source + val[j]
        sz = pickle.load(open(path_val, 'rb'))
        samples_val  +=  np.shape(sz.data)[0]
        val_id_before.append(sz.patient_id)
        val_id_after.append([sz.patient_id]*np.shape(sz.data)[0])
        val_label_before.append(sz.seizure_type)
        val_label_after.append([sz.seizure_type]*np.shape(sz.data)[0])
        val_data.append(sz.data)
        
    flat_train_id_after = [item for sublist in train_id_after for item in sublist]   #concatenate the expended patient id
    flat_train_label_after = [item for sublist in train_label_after for item in sublist] #concatenate the expended train label
    flat_train_data = [item for sublist in train_data for item in sublist]  #concatenate the training data
    train_data = np.reshape(flat_train_data,[samples_train,1,20,24])   #reshape the training data to the format which can be trained by DL algorithm
    
    flat_val_id_after = [item for sublist in val_id_after for item in sublist]
    flat_val_label_after = [item for sublist in val_label_after for item in sublist]
    flat_val_data = [item for sublist in val_data for item in sublist]
    val_data = np.reshape(flat_val_data,[samples_val,1,20,24])
    
    np.save('data_new_train_val_test/train_data_cross_'+str(i+1), train_data)
    np.save('data_new_train_val_test/train_label_before_cross_'+str(i+1), train_label_before)
    np.save('data_new_train_val_test/train_label_after_cross_'+str(i+1), flat_train_label_after)
    np.save('data_new_train_val_test/train_id_before_cross_'+str(i+1), train_id_before)
    np.save('data_new_train_val_test/train_id_after_cross_'+str(i+1), flat_train_id_after)
    np.save('data_new_train_val_test/filename_train_'+str(i+1), filename_train)
    
    np.save('data_new_train_val_test/val_data_cross_'+str(i+1), val_data)
    np.save('data_new_train_val_test/val_label_before_cross_'+str(i+1), val_label_before)
    np.save('data_new_train_val_test/val_label_after_cross_'+str(i+1), flat_val_label_after)
    np.save('data_new_train_val_test/val_id_before_cross_'+str(i+1), val_id_before)
    np.save('data_new_train_val_test/val_id_after_cross_'+str(i+1), flat_val_id_after)
    np.save('data_new_train_val_test/filename_val_'+str(i+1), filename_val)

1279 331 399
1294 316 399
1293 317 399
1275 335 399
1299 311 399


In [17]:
# Check if the training and validation sets still keep the percentage of each category
table = {'ABSZ':0,'CPSZ':0, 'FNSZ':0, 'GNSZ':0, 'SPSZ':0, 'TCSZ':0, 'TNSZ':0}
for item in train_label_before:
    table[item] += 1
    
table


{'ABSZ': 65,
 'CPSZ': 218,
 'FNSZ': 645,
 'GNSZ': 266,
 'SPSZ': 30,
 'TCSZ': 31,
 'TNSZ': 44}

In [24]:
test_data, test_id_before, test_id_after, test_label_before, test_label_after = [],[],[],[],[]
samples_test = 0

for j in range(len(test_all)): #read train data, train label and patient id from the pkl files.
        path_test = source + test_all[j]  #location of the pkl files
        sz = pickle.load(open(path_test, 'rb'))  #load the file
        samples_test  +=  np.shape(sz.data)[0] #the number of EEG clips in each pkl files
        test_id_before.append(sz.patient_id)   #original patient id
        test_id_after.append([sz.patient_id]*np.shape(sz.data)[0])   #expended patient id according to the number of EEG clips in each pkl file
        test_label_before.append(sz.seizure_type)   #original label
        test_label_after.append([sz.seizure_type]*np.shape(sz.data)[0]) #expended training lables according to the number of EEG clips in each pkl file
        test_data.append(sz.data)

In [25]:
len(test_id_before)

399

In [26]:
samples_test

119534

In [41]:
flat_test_id_after = [item for sublist in test_id_after for item in sublist]   #concatenate the expended patient id
flat_test_label_after = [item for sublist in test_label_after for item in sublist] #concatenate the expended train label
flat_test_data = [item for sublist in test_data for item in sublist]  #concatenate the training data
test_data = np.reshape(flat_test_data,[samples_test,1,20,24])

np.save('data_new_train_val_test/test_data', test_data)
np.save('data_new_train_val_test/test_label_before', test_label_before)
np.save('data_new_train_val_test/test_label_after', flat_test_label_after)
np.save('data_new_train_val_test/test_id_before', test_id_before)
np.save('data_new_train_val_test/test_id_afte', flat_test_id_after)
np.save('data_new_train_val_test/filename_test', test_all)

# Process the data for majority voting

In [29]:
infile = open('cv_split_5_fold_seizure_wise_v1.4.0.pkl','rb') #use the data prepared for cross validation
new_dict = pickle.load(infile)
infile.close()

source  = 'processed_2/v1.4.0/fft/fft_seizures_wl1_ws_0.25_sf_250_fft_min_1_fft_max_24/' #load the data from this folder

for i in range(5):
    accumulated_val = [0]
    each_lenth_val = []
    
    val =  new_dict[str(i+1)]['val']      #read the validation data for the 5-fold cross validation
    val = list(set(val) - set(test_all))
    print(len(val), len(test_all))
    
    samples_val = 0
            
    for j in range(len(val)):  #read validation data, validation label and patient id from the pkl files.
        path_val = source + val[j]
        sz = pickle.load(open(path_val, 'rb'))
        each_lenth_val.append(np.shape(sz.data)[0])
        samples_val  +=  np.shape(sz.data)[0]
        accumulated_val.append(samples_val)
    
    accumulated_val = accumulated_val[:-1]
    np.save('data_new_train_val_test/accumulated_val_'+str(i+1), accumulated_val)
    np.save('data_new_train_val_test/each_lenth_val_'+str(i+1), each_lenth_val)


331 399
316 399
317 399
335 399
311 399


In [30]:
samples_test = 0
each_lenth_test, accumulated_test = [], [0]

for j in range(len(test_all)):  #read validation data, validation label and patient id from the pkl files.
    path_test = source + test_all[j]
    sz = pickle.load(open(path_test, 'rb'))
    each_lenth_test.append(np.shape(sz.data)[0])
    samples_test  +=  np.shape(sz.data)[0]
    accumulated_test.append(samples_test)

accumulated_test = accumulated_test[:-1]
np.save('data_new_train_val_test/accumulated_test', accumulated_test)
np.save('data_new_train_val_test/each_lenth_test', each_lenth_test)

In [31]:
len(test_all)

399

In [32]:
len(accumulated_test)

399

In [33]:
len(each_lenth_test)

399

In [34]:
np.sum(each_lenth_test) == accumulated_test[-1] + each_lenth_test[-1]

True

In [35]:
accumulated_test[-1] + each_lenth_test[-1]

119534

In [36]:
np.sum(each_lenth_test)

119534

In [37]:
np.sum(each_lenth_val) == accumulated_val[-1] + each_lenth_val[-1]

True

In [38]:
list(set(train_id_before).difference(set(test_id_before)))

['00006986',
 '00005347',
 '00004049',
 '00001052',
 '00006139',
 '00004087',
 '00008029',
 '00003884',
 '00004456',
 '00007092',
 '00003401',
 '00010421',
 '00000820',
 '00004151',
 '00009232',
 '00001479',
 '00005044',
 '00000975',
 '00005512',
 '00002427',
 '00002521',
 '00008615',
 '00002445',
 '00005625',
 '00007095',
 '00000574',
 '00003053',
 '00010861',
 '00009158',
 '00001753',
 '00000006',
 '00004523',
 '00006811',
 '00008480',
 '00000906',
 '00001795',
 '00009050',
 '00010547',
 '00004596',
 '00005346',
 '00009104',
 '00008552',
 '00006103',
 '00001548',
 '00008092',
 '00007797',
 '00007130',
 '00006535',
 '00002707',
 '00007170',
 '00000775']